In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('Practise').getOrCreate()

In [8]:
## read data set
df=spark.read.csv('test3.csv',header=True,inferSchema=True)

In [9]:
## show df
df.show()

+--------+----+----------+------+
|    Name| Age|Experience|Salary|
+--------+----+----------+------+
|Giuseppe|  23|        10| 30000|
|   Mario|  45|         8| 25000|
|   Fabio|  65|         4| 20000|
|    Luca|  24|         3| 15000|
| Antonio|  26|         1| 23000|
|   Marco|null|      null| 40000|
|    null|  34|        10| 38000|
|    null|  36|      null|  null|
|    null|null|      null|  null|
+--------+----+----------+------+



In [10]:
## drop columns
df.drop('Name').show()

+----+----------+------+
| Age|Experience|Salary|
+----+----------+------+
|  23|        10| 30000|
|  45|         8| 25000|
|  65|         4| 20000|
|  24|         3| 15000|
|  26|         1| 23000|
|null|      null| 40000|
|  34|        10| 38000|
|  36|      null|  null|
|null|      null|  null|
+----+----------+------+



In [11]:
## na.drop with no parameters - even if there's just 1 null entire row will be deleted
## how==any - default
df.na.drop().show()

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|Giuseppe| 23|        10| 30000|
|   Mario| 45|         8| 25000|
|   Fabio| 65|         4| 20000|
|    Luca| 24|         3| 15000|
| Antonio| 26|         1| 23000|
+--------+---+----------+------+



In [13]:
## how==any -> even just 1 null|entire row deleted - default setting
## how==all -> only deletes row where every column is null
df.na.drop(how='all').show()

+--------+----+----------+------+
|    Name| Age|Experience|Salary|
+--------+----+----------+------+
|Giuseppe|  23|        10| 30000|
|   Mario|  45|         8| 25000|
|   Fabio|  65|         4| 20000|
|    Luca|  24|         3| 15000|
| Antonio|  26|         1| 23000|
|   Marco|null|      null| 40000|
|    null|  34|        10| 38000|
|    null|  36|      null|  null|
+--------+----+----------+------+



In [21]:
## threshold
## example thresh=2 - there are at least 2 NON NULL VALUES IN THAT ROW
df.na.drop(how='any',thresh=2).show()

+--------+----+----------+------+
|    Name| Age|Experience|Salary|
+--------+----+----------+------+
|Giuseppe|  23|        10| 30000|
|   Mario|  45|         8| 25000|
|   Fabio|  65|         4| 20000|
|    Luca|  24|         3| 15000|
| Antonio|  26|         1| 23000|
|   Marco|null|      null| 40000|
|    null|  34|        10| 38000|
+--------+----+----------+------+



In [22]:
## subset
## example subset=['Experience'] - only delete rows where "Experience" column has null value
df.na.drop(how='any',subset=['Experience']).show()

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|Giuseppe| 23|        10| 30000|
|   Mario| 45|         8| 25000|
|   Fabio| 65|         4| 20000|
|    Luca| 24|         3| 15000|
| Antonio| 26|         1| 23000|
|    null| 34|        10| 38000|
+--------+---+----------+------+



In [28]:
## filling missing values - in this case it will only work for string types because inferSchema = True
df.na.fill('Missing value').show()

+-------------+----+----------+------+
|         Name| Age|Experience|Salary|
+-------------+----+----------+------+
|     Giuseppe|  23|        10| 30000|
|        Mario|  45|         8| 25000|
|        Fabio|  65|         4| 20000|
|         Luca|  24|         3| 15000|
|      Antonio|  26|         1| 23000|
|        Marco|null|      null| 40000|
|Missing value|  34|        10| 38000|
|Missing value|  36|      null|  null|
|Missing value|null|      null|  null|
+-------------+----+----------+------+



In [30]:
## filling only specific columns
## in this example it will not affect "Salary"
df.na.fill(0,['Age','Experience']).show()

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|Giuseppe| 23|        10| 30000|
|   Mario| 45|         8| 25000|
|   Fabio| 65|         4| 20000|
|    Luca| 24|         3| 15000|
| Antonio| 26|         1| 23000|
|   Marco|  0|         0| 40000|
|    null| 34|        10| 38000|
|    null| 36|         0|  null|
|    null|  0|         0|  null|
+--------+---+----------+------+



In [31]:
## imputer
## in this example for every null value in Age, Experience and Salary
## we will replace null values with entire column mean
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['Age','Experience','Salary'],
    outputCols=["{}_imputed".format(c) for c in ['Age','Experience','Salary']]
).setStrategy("mean")

In [32]:
## add imputation cols to df
imputer.fit(df).transform(df).show()

+--------+----+----------+------+-----------+------------------+--------------+
|    Name| Age|Experience|Salary|Age_imputed|Experience_imputed|Salary_imputed|
+--------+----+----------+------+-----------+------------------+--------------+
|Giuseppe|  23|        10| 30000|         23|                10|         30000|
|   Mario|  45|         8| 25000|         45|                 8|         25000|
|   Fabio|  65|         4| 20000|         65|                 4|         20000|
|    Luca|  24|         3| 15000|         24|                 3|         15000|
| Antonio|  26|         1| 23000|         26|                 1|         23000|
|   Marco|null|      null| 40000|         36|                 6|         40000|
|    null|  34|        10| 38000|         34|                10|         38000|
|    null|  36|      null|  null|         36|                 6|         27285|
|    null|null|      null|  null|         36|                 6|         27285|
+--------+----+----------+------+-------